In [1]:
import feedparser
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

clean_path = "../data/clean/"
raw_path = "../data/raw/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
pdf_path = "../PDF/"

pd.set_option('display.max_colwidth', 255)
pd.set_option('display.max_rows',None)
url = "https://feeds.feedburner.com/Setorth-form45-en"

today = date.today()
year = 2021
mmdd_str = today.strftime('%m%d')
mmdd_str

'0221'

In [2]:
#today = date(2022, 2, 18)
mmdd_str = today.strftime('%m%d')
mmdd_str

'0221'

In [3]:
rss_source = feedparser.parse(url)
f45_number = len(rss_source.entries)
print("Number of F45: ", f45_number)

Number of F45:  63


In [4]:
f45_items = []

for x in range(f45_number):
    f45_content = rss_source.entries[x]
    f45_item = {}
    
    print("\n----------------------------------\n")
    
    print("F45: " + str(x))
    title_ary = f45_content.title.partition(' ')
    f45_item['name'] = title_ary[0].strip() 
    print("Title: ", f45_item['name'])  
    f45_item['year'] = year
    print("Year: ", f45_item['year'])      
    qtr_ary = title_ary[2].partition(' (F45)')
    f45_item['quarter'] = qtr_ary[0][-1]    
    print("Quarter: ", f45_item['quarter'])    
    f45_item['link'] = f45_content.link
    print("Link: ", f45_item['link'])
    f45_item['published'] = f45_content.published
    print("Published: ", f45_item['published'])  
    f45_items.append(f45_item)


----------------------------------

F45: 0
Title:  RBF
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000408440
Published:  Mon, 21 Feb 2022 21:37:19 +0700

----------------------------------

F45: 1
Title:  LEO
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000406400
Published:  Mon, 21 Feb 2022 21:06:27 +0700

----------------------------------

F45: 2
Title:  AGE
Year:  2021
Quarter:  y
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000406230
Published:  Mon, 21 Feb 2022 21:02:33 +0700

----------------------------------

F45: 3
Title:  HREIT
Year:  2021
Quarter:  e
Link:  https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000403580
Published:  Mon, 21 Feb 2022 20:24:31 +0700

----------------------------------

F45: 4
Title:  DREIT
Year:  2021
Quarter:  e
Link:  https://www.set.or.th/set/newsdetails.do?

In [5]:
df = pd.DataFrame(f45_items)
df[['name','year','quarter','published']]

,name,year,quarter,published
0,RBF,2021,y,"Mon, 21 Feb 2022 21:37:19 +0700"
1,LEO,2021,y,"Mon, 21 Feb 2022 21:06:27 +0700"
2,AGE,2021,y,"Mon, 21 Feb 2022 21:02:33 +0700"
3,HREIT,2021,e,"Mon, 21 Feb 2022 20:24:31 +0700"
4,DREIT,2021,e,"Mon, 21 Feb 2022 20:12:33 +0700"
5,TIDLOR,2021,y,"Mon, 21 Feb 2022 20:09:55 +0700"
6,SO,2021,y,"Mon, 21 Feb 2022 19:59:43 +0700"
7,XO,2021,y,"Mon, 21 Feb 2022 19:58:00 +0700"
8,NYT,2021,y,"Mon, 21 Feb 2022 19:45:01 +0700"
9,AU,2021,y,"Mon, 21 Feb 2022 19:40:52 +0700"


In [6]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == 'y','quarter'] = 4
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,50,50,50,50
e,13,13,13,13


In [7]:
df.loc[df.quarter == 'e','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,63,63,63,63


In [8]:
df.loc[df.quarter == 'S','quarter'] = 4
df.groupby(['quarter']).count()

,name,year,link,published
quarter,,,,
4,63,63,63,63


In [10]:
#df.iloc[44,2] = [3]
df.loc[df.quarter == '1','year'] = 2022
#df.replace({"quarter" : "1"}, "3", inplace=True)
df.groupby(['year']).count()

,name,quarter,link,published
year,,,,
2021,32,32,32,32


In [9]:
df.shape

(63, 5)

In [10]:
### First equals to latest published pdf file
df = df.drop_duplicates(subset='name', keep='first')
df.shape

(56, 5)

In [11]:
file_name = 'F45-RAW-' + mmdd_str + '.csv'
raw_file = raw_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df[['name','year','quarter','published']].to_csv(output_file, header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(box_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(one_file,    header=True, index=False, sep=',')
df[['name','year','quarter','published']].to_csv(raw_file,    header=True, index=False, sep=',')

In [12]:
sql = '''
SELECT *
FROM exempts
ORDER BY name'''
df_exempts = pd.read_sql(sql, conlt)
df_exempts.shape[0]

299

In [13]:
df_merge = pd.merge(df, df_exempts, on='name', how='outer', indicator=True)
df_merge.shape

(338, 7)

### Tickers that won't be input

In [14]:
in_exempts = df_merge.loc[
    df_merge['_merge'] == 'both',
    ['name','year','quarter','published','link']
]
in_exempts.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
2,AGE,2021.0,4,"Mon, 21 Feb 2022 21:02:33 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000406230
7,XO,2021.0,4,"Mon, 21 Feb 2022 19:58:00 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000401450
9,AU,2021.0,4,"Mon, 21 Feb 2022 19:40:52 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000400380
12,KBS,2021.0,4,"Mon, 21 Feb 2022 19:20:26 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399020
13,SCG,2021.0,4,"Mon, 21 Feb 2022 19:05:02 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000397670
24,SABINA,2021.0,4,"Mon, 21 Feb 2022 17:38:03 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000384910
26,PG,2021.0,4,"Mon, 21 Feb 2022 17:34:12 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000384250
33,VPO,2021.0,4,"Mon, 21 Feb 2022 17:11:15 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000378930
35,TNL,2021.0,4,"Mon, 21 Feb 2022 17:04:49 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000377110
37,TMI,2021.0,4,"Mon, 21 Feb 2022 17:00:38 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000376140


In [15]:
in_exempts.sort_values(by=['published'],ascending=[False]).shape[0]

17

In [16]:
df_out = df_merge.loc[
    df_merge['_merge'] == 'left_only',
    ['name','year','quarter','published','link']
]
df_out.sort_values(by=['published'],ascending=[False])

,name,year,quarter,published,link
0,RBF,2021.0,4,"Mon, 21 Feb 2022 21:37:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000408440
1,LEO,2021.0,4,"Mon, 21 Feb 2022 21:06:27 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000406400
3,HREIT,2021.0,4,"Mon, 21 Feb 2022 20:24:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000403580
4,DREIT,2021.0,4,"Mon, 21 Feb 2022 20:12:33 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402380
5,TIDLOR,2021.0,4,"Mon, 21 Feb 2022 20:09:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402240
6,SO,2021.0,4,"Mon, 21 Feb 2022 19:59:43 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000401530
8,NYT,2021.0,4,"Mon, 21 Feb 2022 19:45:01 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000400680
10,CPANEL,2021.0,4,"Mon, 21 Feb 2022 19:34:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399900
11,TQR,2021.0,4,"Mon, 21 Feb 2022 19:21:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399050
14,NCAP,2021.0,4,"Mon, 21 Feb 2022 18:50:38 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000396050


In [17]:
#df_out = df_out.drop(df_out.index[df_out['name'] == "SCC"])
df_out.shape[0]

39

In [18]:
sql = '''
SELECT *
FROM tickers
ORDER BY name'''
df_tickers = pd.read_sql(sql, conlt)
df_tickers.shape

(403, 9)

In [19]:
df_merge2 = pd.merge(df_out, df_tickers, on='name', how='outer', indicator=True)
df_merge2.shape

(422, 14)

### There are no ticker records

In [20]:
df_merge2.loc[
    df_merge2['_merge'] == 'left_only',
    ['name','year','quarter','published','link','id','market']
]

,name,year,quarter,published,link,id,market
1,LEO,2021.0,4,"Mon, 21 Feb 2022 21:06:27 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000406400,NaN,NaN
3,DREIT,2021.0,4,"Mon, 21 Feb 2022 20:12:33 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402380,NaN,NaN
5,SO,2021.0,4,"Mon, 21 Feb 2022 19:59:43 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000401530,NaN,NaN
7,CPANEL,2021.0,4,"Mon, 21 Feb 2022 19:34:13 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399900,NaN,NaN
9,NCAP,2021.0,4,"Mon, 21 Feb 2022 18:50:38 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000396050,NaN,NaN
10,MJLF,2021.0,4,"Mon, 21 Feb 2022 18:38:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000394380,NaN,NaN
12,CTARAF,2021.0,4,"Mon, 21 Feb 2022 18:31:11 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000393450,NaN,NaN
13,TPLAS,2021.0,4,"Mon, 21 Feb 2022 18:19:57 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000391480,NaN,NaN
15,MNRF,2021.0,4,"Mon, 21 Feb 2022 17:57:35 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000388940,NaN,NaN
16,M-PAT,2021.0,4,"Mon, 21 Feb 2022 17:55:32 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000388490,NaN,NaN


### There are ticker records

In [21]:
df_out2 = df_merge2.loc[
    df_merge2['_merge'] == 'both',
    ['name','year','quarter','published','link','id','market']
]
df_out2

,name,year,quarter,published,link,id,market
0,RBF,2021.0,4,"Mon, 21 Feb 2022 21:37:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000408440,703.0,SET100 / SETCLMV / SETWB
2,HREIT,2021.0,4,"Mon, 21 Feb 2022 20:24:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000403580,211.0,SET
4,TIDLOR,2021.0,4,"Mon, 21 Feb 2022 20:09:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402240,723.0,SET50
6,NYT,2021.0,4,"Mon, 21 Feb 2022 19:45:01 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000400680,336.0,sSET / SETTHSI
8,TQR,2021.0,4,"Mon, 21 Feb 2022 19:21:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399050,716.0,mai
11,AIT,2021.0,4,"Mon, 21 Feb 2022 18:36:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000394130,11.0,sSET
14,RJH,2021.0,4,"Mon, 21 Feb 2022 18:14:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000390540,399.0,sSET
17,JASIF,2021.0,4,"Mon, 21 Feb 2022 17:48:07 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000386970,234.0,SET
18,CKP,2021.0,4,"Mon, 21 Feb 2022 17:41:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000385920,107.0,SET100 / SETCLMV / SETTHSI
19,SNP,2021.0,4,"Mon, 21 Feb 2022 17:35:50 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000384470,694.0,SET


In [22]:
df_out2 = df_out2[df_out2.year.notnull()]
df_out2

,name,year,quarter,published,link,id,market
0,RBF,2021.0,4,"Mon, 21 Feb 2022 21:37:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000408440,703.0,SET100 / SETCLMV / SETWB
2,HREIT,2021.0,4,"Mon, 21 Feb 2022 20:24:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000403580,211.0,SET
4,TIDLOR,2021.0,4,"Mon, 21 Feb 2022 20:09:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402240,723.0,SET50
6,NYT,2021.0,4,"Mon, 21 Feb 2022 19:45:01 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000400680,336.0,sSET / SETTHSI
8,TQR,2021.0,4,"Mon, 21 Feb 2022 19:21:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399050,716.0,mai
11,AIT,2021.0,4,"Mon, 21 Feb 2022 18:36:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000394130,11.0,sSET
14,RJH,2021.0,4,"Mon, 21 Feb 2022 18:14:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000390540,399.0,sSET
17,JASIF,2021.0,4,"Mon, 21 Feb 2022 17:48:07 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000386970,234.0,SET
18,CKP,2021.0,4,"Mon, 21 Feb 2022 17:41:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000385920,107.0,SET100 / SETCLMV / SETTHSI
19,SNP,2021.0,4,"Mon, 21 Feb 2022 17:35:50 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000384470,694.0,SET


In [23]:
df_out2['year'] = df_out2['year'].astype(int)
df_out2['quarter'] = df_out2['quarter'].astype(int)
df_out2

,name,year,quarter,published,link,id,market
0,RBF,2021,4,"Mon, 21 Feb 2022 21:37:19 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000408440,703.0,SET100 / SETCLMV / SETWB
2,HREIT,2021,4,"Mon, 21 Feb 2022 20:24:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000403580,211.0,SET
4,TIDLOR,2021,4,"Mon, 21 Feb 2022 20:09:55 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402240,723.0,SET50
6,NYT,2021,4,"Mon, 21 Feb 2022 19:45:01 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000400680,336.0,sSET / SETTHSI
8,TQR,2021,4,"Mon, 21 Feb 2022 19:21:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399050,716.0,mai
11,AIT,2021,4,"Mon, 21 Feb 2022 18:36:08 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000394130,11.0,sSET
14,RJH,2021,4,"Mon, 21 Feb 2022 18:14:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000390540,399.0,sSET
17,JASIF,2021,4,"Mon, 21 Feb 2022 17:48:07 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000386970,234.0,SET
18,CKP,2021,4,"Mon, 21 Feb 2022 17:41:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000385920,107.0,SET100 / SETCLMV / SETTHSI
19,SNP,2021,4,"Mon, 21 Feb 2022 17:35:50 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000384470,694.0,SET


In [24]:
file_name = 'F45-CLEAN-' + mmdd_str + '.csv'
clean_file = clean_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(output_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(clean_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(box_file, header=True, index=False, sep=',')
df_out2[['name','year','quarter','published','market']].sort_values(['published'],ascending=[False]).to_csv(one_file, header=True, index=False, sep=',')

In [25]:
sql = '''
SELECT * 
FROM epss
WHERE year = 2021'''
df_epss = pd.read_sql(sql, conlt)
df_epss.shape

(1108, 14)

In [26]:
df_merge3 = pd.merge(df_out2, df_epss, on=['name','year','quarter'], how='outer', indicator=True)
df_merge3.shape

(1120, 19)

### Already input, display profit amt & eps to check with new F45

In [27]:
df_merge3[df_merge3['_merge'] == 'both']

,name,year,quarter,published,link,id_x,market,id_y,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date,_merge
1,HREIT,2021,4,"Mon, 21 Feb 2022 20:24:31 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000403580,211.0,SET,21185.0,332692.0,226266.0,892581.0,609075.0,0.3960,0.1802,1.0624,0.7250,211.0,2022-02-21,both
6,RJH,2021,4,"Mon, 21 Feb 2022 18:14:42 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000390540,399.0,sSET,21182.0,283360.0,144491.0,1011958.0,412495.0,0.9400,0.4800,3.3700,1.3700,399.0,2022-02-21,both
7,JASIF,2021,4,"Mon, 21 Feb 2022 17:48:07 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000386970,234.0,SET,21183.0,2502705.0,2602275.0,8029511.0,9100688.0,0.3128,0.3253,1.0037,1.1376,234.0,2022-02-21,both
8,CKP,2021,4,"Mon, 21 Feb 2022 17:41:09 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000385920,107.0,SET100 / SETCLMV / SETTHSI,21186.0,122540.0,7798.0,2179013.0,404714.0,0.0150,0.0010,0.2680,0.0500,107.0,2022-02-21,both
15,TISCO,2021,4,"Mon, 21 Feb 2022 13:17:11 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000356880,531.0,SET50 / SETHD / SETTHSI,21078.0,1791340.0,1636472.0,6781472.0,6063483.0,2.2400,2.0400,8.4700,7.5700,531.0,2022-01-14,both
16,MAKRO,2021,4,"Mon, 21 Feb 2022 08:47:51 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000339480,283.0,SET,21171.0,9094011.0,2129873.0,13686726.0,6562667.0,1.4200,0.4500,2.3800,1.3700,283.0,2022-02-18,both
17,NER,2021,4,"Mon, 21 Feb 2022 08:42:18 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000338770,680.0,sSET,21184.0,604563.0,421572.0,1850190.0,858677.0,0.3640,0.2710,1.1300,0.5550,680.0,2022-02-21,both
19,TIPCO,2021,4,"Mon, 21 Feb 2022 07:41:14 +0700",https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000335730,530.0,sSET,21168.0,120485.0,194036.0,428602.0,552925.0,0.2500,0.4100,0.8900,1.1500,530.0,2022-02-18,both


In [28]:
df_inp2 = df_merge3[df_merge3['_merge'] == 'left_only']
df_inp3 = df_inp2.copy()
df_inp3.shape

(12, 19)

In [29]:
df_inp3['year'] = df_inp3['year'].astype(str)
df_inp3['quarter'] = df_inp3['quarter'].astype(str)
inp3_str = df_inp3.name+' '+df_inp3.year+' '+df_inp3.quarter+' '+df_inp3.link
inp3_str

0        RBF 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000408440
2     TIDLOR 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000402240
3        NYT 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000400680
4        TQR 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000399050
5        AIT 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000394130
9        SNP 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000384470
10        PB 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000382460
11      BSBM 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000381090
12     SHANG 2021 4 https://www.set.or.th/set/newsdetails.do?language=en&country=US&newsId=16454000378490
13       SVH 2021 4 https://www.set.or.th/set/